In [183]:
import numpy as np
import pandas as pd
from collections import Counter

In [184]:
base = "rhinohorn"

In [185]:
#df=pd.read_csv(f"./{base}/{base}_filtered_tweets.csv")
df=pd.read_csv(f"./{base}/{base}_tweets_classifier0.csv")


In [186]:
df.shape

(104042, 6)

In [187]:
df.head()

,Unnamed: 0.1,Unnamed: 0,tweet_link,username,tweet_content,classification
0,0,0,/FindingMirrors/status/1793662749027045530#m,Finding mirrors 🪞,"47.2 Inch Full Length Mirror with Storage, Wal...",0.276379
1,1,1,/FindingMirrors/status/1793662746179039715#m,Finding mirrors 🪞,White Dinosaur Mirror Set of Two 9”dlvr.it/T7J1cd,0.464151
2,2,2,/FindingMirrors/status/1793662743826071928#m,Finding mirrors 🪞,Two (2) Dinosaur Mirrors - Blue And Green Glow...,0.352693
3,3,3,/FindingMirrors/status/1793662739241746542#m,Finding mirrors 🪞,Rare 19th Century French Rococo Reverse Decoup...,0.407587
4,4,4,/FindingMirrors/status/1793662736645403037#m,Finding mirrors 🪞,Delicate DIY Cloud Tulip LED Night Light Mirro...,0.285308


In [188]:
import re
def extract_handle(link):
    match = re.search(r'^/([^/]+)/status/',link)
    if match:
        return "@"+match.group(1)
    return None
                      

In [189]:
df['handle'] = df['tweet_link'].apply(extract_handle)

In [190]:
df.head()
print(len(df))

104042


In [191]:
df['username']=df['username'].astype('str')
df['handle'] = df['handle'].astype('str')

In [192]:
def create_tweet_dict(group):
    return[{'tweet_link': row['tweet_link'], 'tweet_content': row['tweet_content'],'tweet_classification':row['classification']} for index, row in group.iterrows()]

In [193]:
grouped = df.groupby(['handle','username'])

In [194]:
grouped

In [195]:
new_data = {
    'handle' : [],
    'username': [],
    'tweets':[],
    'num_flagged_tweets': []
}

In [196]:
def calc_num_flagged_tweets(tweets):
    count = 0
    for tweet in tweets:
        if(tweet['tweet_classification']>=0.9):
            count+=1
    return count

for (handle, username), group in grouped:
    new_data['handle'].append(handle)
    new_data['username'].append(username)
    new_data['tweets'].append(create_tweet_dict(group))
    new_data['num_flagged_tweets'].append(calc_num_flagged_tweets(new_data['tweets'][-1]))


In [197]:
new_df = pd.DataFrame(new_data)

In [198]:
new_df.head()

,handle,username,tweets,num_flagged_tweets
0,@0xMises,Anon econ,[{'tweet_link': '/0xMises/status/1767702797405...,0
1,@1147_001,1147 001,[{'tweet_link': '/1147_001/status/179366665610...,1
2,@19myers87,myers,[{'tweet_link': '/19myers87/status/17437879227...,0
3,@1ferhatk,ferhat,[{'tweet_link': '/1ferhatk/status/179365571867...,0
4,@1yellowjonquil,🐝 Bee KennedyShanahan Kind,[{'tweet_link': '/1yellowjonquil/status/179255...,0


In [199]:
import pandas as pd
df1 = new_df
df2 = pd.read_csv(f'{base}/{base}_feature_engineered.csv')
df2.head()

,Unnamed: 0.1,Unnamed: 0,join_date,replies,tweets,handle,tweet_count,followers_count,following_count,username,...,time_series,mean,median,std_dev,min,max,25th_percentile,75th_percentile,num_scraped_tweets,avg_tweet_length
0,0,0,1:48 PM - 1 Nov 2023,[],[{'tweet_link': '/FindingMirrors/status/179366...,@FindingMirrors,291431.0,63,2,Finding mirrors 🪞,...,"[(1716422400, 200)]",200.000000,200.0,2.083211e+17,200,200,200.0,200.0,200,82.700000
1,1,1,4:48 AM - 11 May 2018,[{'tweet_link': '/ohmybeautybank/status/179367...,[{'tweet_link': '/ohmybeautybank/status/173928...,@ohmybeautybank,1886061.0,"1,627,230","2,817",📌 GIVEAWAY ON PINNED✨,...,"[(1703462400, 2), (1716422400, 192)]",97.000000,97.0,4.155482e+17,2,192,49.5,144.5,194,110.712871
2,2,2,8:14 AM - 16 Jan 2020,[],[{'tweet_link': '/HelloMaxHello/status/1794661...,@HelloMaxHello,1134322.0,"3,415","3,187",HelloMax,...,"[(1716681600, 200)]",200.000000,200.0,2.083840e+17,200,200,200.0,200.0,200,98.030000
3,3,3,1:04 AM - 27 Jun 2017,[],[{'tweet_link': '/1147_001/status/179366665610...,@1147_001,1495596.0,"8,073","7,371",1147 001,...,"[(1716422400, 200)]",200.000000,200.0,2.083211e+17,200,200,200.0,200.0,200,114.480000
4,4,4,11:54 PM - 26 Mar 2015,[],[{'tweet_link': '/followbacksen/status/1093553...,@followbacksen,1323793.0,"4,926","4,212",john edwardsen,...,"[(1549497600, 2), (1579910400, 168), (15800832...",33.333333,8.0,8.963543e+17,2,168,3.0,11.5,200,85.060000


In [200]:
# keys = ['following_count', 
#         'followers_count',
#         'post_freq',
#         'num_scraped_tweets',
#         'mean',
#         'median',
#         'std_dev',
#         'min',
#         'max',
#         '25th_percentile',
#         '75th_percentile'
#        ]
keys = ['followers_count', 'post_freq', 'std_dev', 'avg_tweet_length', 'num_scraped_tweets']
for key in keys:
    fmap = df2.set_index('handle')[key].to_dict()
    df1[key] = df1['handle'].map(fmap)

In [201]:
temp =df1.copy()

In [202]:
df1.head()

,handle,username,tweets,num_flagged_tweets,followers_count,post_freq,std_dev,avg_tweet_length,num_scraped_tweets
0,@0xMises,Anon econ,[{'tweet_link': '/0xMises/status/1767702797405...,0,101,5.336634,2.068347e+18,76.330000,2
1,@1147_001,1147 001,[{'tweet_link': '/1147_001/status/179366665610...,1,"8,073",593.019826,2.083211e+17,114.480000,200
2,@19myers87,myers,[{'tweet_link': '/19myers87/status/17437879227...,0,4,0.211724,2.008950e+18,71.414634,20
3,@1ferhatk,ferhat,[{'tweet_link': '/1ferhatk/status/179365571867...,0,222,43.905759,6.767844e+17,86.125000,38
4,@1yellowjonquil,🐝 Bee KennedyShanahan Kind,[{'tweet_link': '/1yellowjonquil/status/179255...,0,138,51.746032,8.726711e+17,79.210526,76


In [203]:
print(len(temp))

1467


In [204]:
#temp.to_csv(f'{base}/{base}_filtered_usercount.csv')
temp.to_csv(f'{base}/{base}_usercount_classifier0.csv')
